In [1]:
from IPython.display import Image
from IPython.core.display import HTML 

import warnings
warnings.filterwarnings('ignore')

## [DSO] Machine Learning Seminar Vol.7
2020-08-xx  
SKUE

In [2]:
Image(url="https://image.yodobashi.com/product/100/000/009/002/935/105/100000009002935105_10204.jpg")

+ STEP1
 + https://ai.stanford.edu/~amaas/data/sentiment/  
 からダウンロードして、ファイルを適当なフォルダに格納する。
 
+ STEP2  
 + そのディレクトリに移動して、ターミナルで  
 tar -zxf aclImdb_v1.tar.gz  
 を実行する。
